1D Maxwell equations
===========

Maxwell's equations read (speed of light = 1):

$$
\partial_t\vec B = -\mathrm{rot}\vec E\\
\partial_t\vec E = \mathrm{rot}\vec B
$$

Writing all the components we get for magnetic field:

$$
\partial_t B_x = -\left(\partial_y E_z - \partial_z E_y \right)\\
\partial_t B_y = -\left(\partial_y E_x - \partial_z E_z \right)\\
\partial_t B_z = -\left(\partial_y E_y - \partial_z E_x \right)\\
$$

And for electric field:
$$
\partial_t E_x = \left(\partial_y B_z - \partial_z B_y \right)\\
\partial_t E_y = \left(\partial_y B_x - \partial_z B_z \right)\\
\partial_t E_z = \left(\partial_y B_y - \partial_z B_x \right)\\
$$

Assume 1D geometry, i.e. everything depends only on $t$ and $x$ (infinitely large electromagnetic waves in $y$ and $z$ dimensions). Then we get:

$$
\partial_t B_y = \partial_x E_z\\
\partial_t B_z = -\partial_x E_y
$$

and 

$$
\partial_t E_y= - \partial_x B_z\\
\partial_t E_z= \partial_x B_y
$$

One can see, that these 4 equations can be split into two parts (they are called polarizations of the wave): $E_y, B_z$ (p-polarization) and $E_z, B_y$ (s-polarization)

We get two separate and completely independent sets of equations:

1. P-polarization
-----------------

$$
\partial_t E_y= - \partial_x B_z\\
\partial_t B_z = -\partial_x E_y
$$

2. S-polarization
-----------------

$$
\partial_t E_z= \partial_x B_y\\
\partial_t B_y = \partial_x E_z
$$


Task 1
------

Before proceeding with numerical solution of these equations (let us for simplicity and saving time only deal with p-polarization today) think of a numerical scheme how to solve equations. There are many numerical methods to do this, but can you without looking further in the text come up with one? 

Tip: if you understand what a fractional (1/2) time or spatial step means - you can use it.

---
Without exploring on stability, convergence, approx. order and etc, I write the following scheme
\begin{align}
    &\frac{\Delta x}{\Delta t} (E_i^{n+1/2} - E_i^n) = - B_{i+1/2}^n + B_i^{n-1/2}\\
    &\frac{\Delta x}{\Delta t} (B_i^n - B_i^{n-1/2}) = - E_i^n + E_{i-1/2}^n
\end{align}

Numerical scheme
----------------

Let's look again at the equations for p-polarization:
$$
\partial_t E_y= - \partial_x B_z\\
\partial_t B_z = -\partial_x E_y
$$

Lets sum equations and substract equations and use notation $F_+=E_y+B_z$ and $F_-=E_y-B_z$:

$$
\left(\partial_t + \partial_x\right)F_+ = 0\\
\left(\partial_t - \partial_x\right)F_- = 0
$$

Task 2
------

One of the new fields $F_+$ and $F_-$ corresponds to the wave going from left to right and one to the wave going from right to left. Which is which?

**Answer:**  
$F_+$ - from right to left  
$F_-$ - from left to right

Task 3
------

Derive the numerical scheme (consider the time step equals to spatial step, i.e. $\Delta t$ = $\Delta x$):

$$
F_{+,i}^{n+1} = F_{+,i-1}^n\\
F_{-,i}^{n+1} = F_{-,i+1}^n
$$

**Derivation**
$$
    \frac{F_{+,i}^{n+1} - F_{+,i}^n}{dt} + \frac{F_{+,i}^n - F_{+,i-1}^n}{dx} = 0
$$

Since, we have chosen $\Delta t = \Delta x$, we obtain 1st equation  
In the same way, we get 2nd one

Task 4
------

Consider 1D computational domain of length $L=2\pi \cdot 40$ (you can vary the numbers later), consider only  $F_+$

$$
\Delta t = \Delta x = \frac{2\pi}{1000}
$$

Initially there are no fields in the computational domain.

Boundary condition:

$$
F_+(x=0,t)=sin(t),\quad \mathrm{if}\quad t<2\pi\cdot 10\\
F_+(x=0,t)=0, \mathrm{otherwise}
$$

Write a code (for example in Python for prototyping) that solves 1D Maxwell equations.


In [3]:
import numpy as np

In [55]:
# For the sake of simplicity and power of representativeness in python prototype "values" are chosen
# as boundary condition values
N = 5
values = np.arange(-N+1,N)
F = np.empty(N * N,dtype=int)
print("wave magnitude at x=0 in time moments from (-4,4):")
print(*values)
print()

print("field values in the square area (x,t) in (0,4) x (0,4)")
for i in range(N):
    for j in range(N):
        F[j + N * i] = values[i - j + N - 1]

for i in range(N,0,-1):
    print(*F[(i-1) * N:i * N], sep='\t')

wave magnitude at x=0 in time moments from (-4,4):
-4 -3 -2 -1 0 1 2 3 4

fied values in the square area (x,t) in (0,4) x (0,4)
4	3	2	1	0
3	2	1	0	-1
2	1	0	-1	-2
1	0	-1	-2	-3
0	-1	-2	-3	-4


Task 5
------

Write a code in C/C++ and parallelize it using OpenMP.

In [56]:
# written in a separate file also

```cpp
#include <cstdio>
#include <cmath>
#include <omp.h>
#include <cstdlib>
#define pi 3.141592


int main(int argc, char **argv) {
    int   mult = std::atoi(argv[1]);
    float L = 8 * pi * mult;
    int   N = std::atoi(argv[2]);
    float dx = L / N;
    int   bp = 2 * pi / dx * mult;

// create an array for the solution
    float * F = new float [N * N];

// generate an array containing the boundary values
    float * values = new float [2 * N - 1];

    double t0 = omp_get_wtime();
    #pragma omp parallel
    {
        #pragma omp for nowait
        for (int i=0; i<N+bp; ++i) {
            values[i] = std::sin((-N + 1 + i) * dx);
        }
        
        #pragma omp for
        for (int i=N+bp; i<2*N; ++i) {
            values[i] = 0; 
        }
    }
    double t1 = omp_get_wtime();

    printf("time spent on bc initialization : %fs\n", t1 - t0);

// fill the solution array
    #pragma omp parallel for collapse(2) 
    for (int i=0; i<N; ++i) {
        for (int j=0; j<N; ++j) {
            F[j + N * i] = values[i - j + N - 1];
        }
    } 

    printf("time spent on filling the sol. array : %.3fs\n", 
            omp_get_wtime() - t1);

    FILE * fout = fopen("output.bin", "wb");
    fwrite(F, sizeof(float), N * N, fout);
    fclose(fout);

// deallocate memory
    delete [] values;
    delete [] F;
}

```

**See visualization in the separate file**

# Conclusion

With the use of OMP I have reduced the program runtime several times (7 ~ 15; 32 threads)